In [1]:
import pandas as pd
import numpy as np

# Load main data
data = pd.read_excel("data/Analytics and Forecasting - Calls Data.xlsx", sheet_name="Data")

data['DAY'] = pd.to_datetime(data['DAY'])
data['DATE'] = data['DAY'].dt.date
data['MONTH'] = data['DAY'].dt.month
data['MONTH_NAME'] = data['DAY'].dt.strftime('%B')
data['QUARTER'] = data['DAY'].dt.quarter
data['TOTAL_CALLS'] = data['CALLS HANDLED'] + data['CALLS MISSED']


In [2]:
# Monthly totals
monthly = (
    data.groupby(['MONTH', 'MONTH_NAME'])
        .agg(total_calls=('TOTAL_CALLS','sum'))
        .reset_index()
        .sort_values('MONTH')
)

monthly


,MONTH,MONTH_NAME,total_calls
0,1,January,525312.0
1,2,February,509694.0
2,3,March,424036.5
3,4,April,284634.0
4,5,May,325135.5
5,6,June,309541.5
6,7,July,385971.0
7,8,August,406243.5
8,9,September,386244.0
9,10,October,386946.0


In [3]:
# Quarterly totals
quarterly = (
    data.groupby('QUARTER')
        .agg(total_calls=('TOTAL_CALLS','sum'))
        .reset_index()
        .sort_values('QUARTER')
)

quarterly


,QUARTER,total_calls
0,1,1459042.5
1,2,919311.0
2,3,1178458.5
3,4,1247832.0


In [4]:
q1 = quarterly.loc[quarterly['QUARTER']==1, 'total_calls'].iloc[0]
q2 = quarterly.loc[quarterly['QUARTER']==2, 'total_calls'].iloc[0]
q3 = quarterly.loc[quarterly['QUARTER']==3, 'total_calls'].iloc[0]
q4 = quarterly.loc[quarterly['QUARTER']==4, 'total_calls'].iloc[0]

q1_q2 = (q2 - q1) / q1 * 100
q2_q3 = (q3 - q2) / q2 * 100
q3_q4 = (q4 - q3) / q3 * 100

avg_growth = (q1_q2 + q2_q3 + q3_q4) / 3

q1_q2, q2_q3, q3_q4, avg_growth


(-36.992171235587726,
 28.18931786957841,
 5.886800426149924,
 -0.9720176466197973)

In [5]:
# Use 2024 Q1 & Q2 as baseline
q1_2024_months = monthly[monthly['MONTH'].isin([1,2,3])]
q2_2024_months = monthly[monthly['MONTH'].isin([4,5,6])]

q1_2024_total = q1_2024_months['total_calls'].sum()
q2_2024_total = q2_2024_months['total_calls'].sum()

growth = avg_growth / 100  # convert to decimal

q1_2025_base = q1_2024_total * (1 + growth)
q2_2025_base = q2_2024_total * (1 + growth)

q1_2025_base, q2_2025_base


(1444860.3494283175, 910375.134852683)

In [6]:
q1_adj_factor = 1.03   # +3% events
q2_adj_factor = 1.08   # +8% events

q1_2025_event = q1_2025_base * q1_adj_factor
q2_2025_event = q2_2025_base * q2_adj_factor

h1_2025_event = q1_2025_event + q2_2025_event

q1_2025_event, q2_2025_event, h1_2025_event


(1488206.159911167, 983205.1456408978, 2471411.305552065)

In [7]:
# Spread Q1 forecast across Jan/Feb/Mar using 2024 proportions
q1_2024_months = q1_2024_months.copy()
q1_2024_months['share'] = q1_2024_months['total_calls'] / q1_2024_total
q1_2025_monthly = q1_2024_months[['MONTH','MONTH_NAME','share']].copy()
q1_2025_monthly['forecast_calls'] = q1_2025_event * q1_2025_monthly['share']

# Q2
q2_2024_months = q2_2024_months.copy()
q2_2024_months['share'] = q2_2024_months['total_calls'] / q2_2024_total
q2_2025_monthly = q2_2024_months[['MONTH','MONTH_NAME','share']].copy()
q2_2025_monthly['forecast_calls'] = q2_2025_event * q2_2025_monthly['share']

q1_2025_monthly, q2_2025_monthly


(   MONTH MONTH_NAME     share  forecast_calls
 0      1    January  0.360039   535812.050900
 1      2   February  0.349335   519881.874908
 2      3      March  0.290627   432512.234104,
    MONTH MONTH_NAME     share  forecast_calls
 3      4      April  0.309617   304416.691875
 4      5        May  0.353673   347733.135610
 5      6       June  0.336710   331055.318156)

In [8]:
# Assumptions (you can tweak later): 6 calls per agent-hour, productivity factor 0.9, 3 shifts per day, 8 hours per shift

calls_per_agent_hour = 6
productivity = 0.9
eff_calls_per_hour = calls_per_agent_hour * productivity

# approximate days in Q1/Q2
days_q1 = 90
days_q2 = 91

avg_daily_q1 = q1_2025_event / days_q1
avg_daily_q2 = q2_2025_event / days_q2

hours_q1_per_day = avg_daily_q1 / eff_calls_per_hour
hours_q2_per_day = avg_daily_q2 / eff_calls_per_hour

agents_per_shift_q1 = np.ceil(hours_q1_per_day / 8)
agents_per_shift_q2 = np.ceil(hours_q2_per_day / 8)

total_agents_q1 = agents_per_shift_q1 * 3
total_agents_q2 = agents_per_shift_q2 * 3

avg_daily_q1, avg_daily_q2, total_agents_q1, total_agents_q2


(16535.62399901297, 10804.452149899977, 1149.0, 753.0)

In [9]:
# Q1: keep 58.7% / 41.3%
vendor_a_share_q1 = 0.587
vendor_b_share_q1 = 0.413

vendor_a_agents_q1 = int(total_agents_q1 * vendor_a_share_q1)
vendor_b_agents_q1 = int(total_agents_q1 * vendor_b_share_q1)

# Q2: shift to 60 / 40
vendor_a_share_q2 = 0.60
vendor_b_share_q2 = 0.40

vendor_a_agents_q2 = int(total_agents_q2 * vendor_a_share_q2)
vendor_b_agents_q2 = int(total_agents_q2 * vendor_b_share_q2)

vendor_a_agents_q1, vendor_b_agents_q1, vendor_a_agents_q2


(674, 474, 451)

In [10]:
vendor_a_agents_q1, vendor_b_agents_q1, vendor_a_agents_q2, vendor_b_agents_q2


(674, 474, 451, 301)

In [11]:
# Summary table for H1 2025 forecast & staffing

summary_h1 = pd.DataFrame({
    'quarter': ['Q1 2025', 'Q2 2025'],
    'forecast_calls_event_adj': [round(q1_2025_event), round(q2_2025_event)],
    'avg_daily_calls': [round(avg_daily_q1), round(avg_daily_q2)],
    'total_agents_24x7': [int(total_agents_q1), int(total_agents_q2)],
    'vendor_a_agents': [vendor_a_agents_q1, vendor_a_agents_q2],
    'vendor_b_agents': [vendor_b_agents_q1, vendor_b_agents_q2]
})

summary_h1


,quarter,forecast_calls_event_adj,avg_daily_calls,total_agents_24x7,vendor_a_agents,vendor_b_agents
0,Q1 2025,1488206,16536,1149,674,474
1,Q2 2025,983205,10804,753,451,301


In [12]:
# Export forecast & staffing outputs

q1_2025_monthly.to_csv("data/q1_2025_monthly_forecast.csv", index=False)
q2_2025_monthly.to_csv("data/q2_2025_monthly_forecast.csv", index=False)
summary_h1.to_csv("data/h1_2025_forecast_staffing_summary.csv", index=False)

print("Exported q1_2025_monthly_forecast.csv, q2_2025_monthly_forecast.csv, h1_2025_forecast_staffing_summary.csv")


Exported q1_2025_monthly_forecast.csv, q2_2025_monthly_forecast.csv, h1_2025_forecast_staffing_summary.csv
